In [2]:
import numpy as np
import pandas as pd
from scipy import stats

%precision 3
np.random.seed(1111)

In [3]:
df = pd.read_csv('/Users/yuya/Desktop/Python_Static/python_stat_sample/data/ch11_potato.csv')
sample = np.array(df['重さ'])
sample

array([122.02, 131.73, 130.6 , 131.82, 132.05, 126.12, 124.43, 132.89,
       122.79, 129.95, 126.14, 134.45, 127.64, 125.68])

In [4]:
# 標本平均を求める
s_mean = np.mean(sample)
s_mean

128.4507142857143

In [5]:
# 母平均は130gという仮定で仮説検定を行う
# ※フライドポテトの母集団は正規分布で、その分散は9とする

# 標本平均XがP(X<=x) = 0.05 を満たすxを考える
rv = stats.norm(130, np.sqrt(9/14))
rv.isf(0.95)

128.68118313069039

In [6]:
# 上記の計算により、標本平均が128.68g以下になる確率は
# 5%である事が分かった
# Aさんの抽出した標本平均が128.451gなので、
# これは5%しか起きない珍しい出来事だと考えられる

# 標準化した値で再計算してみる
# 検定統計量
z = (s_mean - 130) / np.sqrt(9/14)
z

-1.932298779026813

In [7]:
# 臨界値を求める
rv = stats.norm()
rv.isf(0.95)

-1.6448536269514722

In [8]:
# 検定統計量 < 臨界値
# だったので、帰無仮説は棄却され、「平均は130gより少ない」という結論になる

# p値を使った仮説検定
# p値を求める
rv.cdf(z)

0.026661319523126635

In [9]:
# 130gより多い・少ない場合の両方を考える
# →両側検定

# フライドポテトの例で有意水準5%の両側検定を行う
z = (s_mean - 130) / np.sqrt(9/14)
z

-1.932298779026813

In [10]:
rv = stats.norm()
rv.interval(0.95)

(-1.959963984540054, 1.959963984540054)

In [11]:
rv.cdf(z) * 2

0.05332263904625327

In [12]:
# 第一種の過誤→帰無仮説が正しいときに、帰無仮説を棄却してしまう過誤
# 第二種の過誤→対立仮説が正しいときに、帰無仮説を採択してしまう過誤

# 今回の例でいうと
# 第一種の過誤→「平均が130g」であるにも関わらず、「平均は130gより少ない」
rv = stats.norm(130, 3)

In [13]:
# 第一種の過誤
# この母集団から14個の標本を抽出して仮説検定を行うという
# 作業を10000回行い、「平均が130g」であるのにも関わらず
# 「平均は130gより少ない」と結論づけてしまう割合を算出
c = stats.norm().isf(0.95)
n_samples = 10000
cnt = 0

for _ in range(n_samples):
    sample_ = np.round(rv.rvs(14), 2)
    s_mean_ = np.mean(sample_)
    z = (s_mean_ - 130) / np.sqrt(9/14)
    if z < c:
        cnt += 1
        
cnt / n_samples

# 第一種の過誤を犯す確率を危険率と呼ぶ
# 危険率は有意水準と一致するので、分析のパラメータ決め次第で
# 制御する事ができる

0.053

In [14]:
# 第二種の過誤
# 実際は「母平均が130gより少ない」にも関わらず「母平均は130gより少ない」
# という結論を得ることができない状況
# →本来検出すべき物を検出できていないので、「検出漏れ」とも呼ぶ
rv = stats.norm(128, 3)
# このケースになる割合を求める
c = stats.norm().isf(0.95)
n_samples = 10000
cnt = 0
for _ in range(n_samples):
    sample_ = np.round(rv.rvs(14), 2)
    s_mean_ = np.mean(sample_)
    z = (s_mean_ - 130) / np.sqrt(9/14)
    if z >= c:
        cnt += 1
cnt / n_samples

# 第二種の過誤を犯す確率には記号βが使われ、1-βを検出力という。
# このβは母集団の情報に依存するため、分析者が制御できない。

0.197

In [15]:
# 正規分布の母平均の仮説検定を実装
# →母平均がある値μではないと主張するための検定
def pmean_test(sample, mean0, p_var, alpha=0.05):
    s_mean = np.mean(sample)
    n = len(sample)
    rv = stats.norm()
    interval = rv.interval(1-alpha)
    
    z = (s_mean - mean0) / np.sqrt(p_var/n)
    if interval[0] <= z <= interval[1]:
        print('帰無仮説を採択')
    else:
        print('帰無仮説を棄却')
    if z < 0:
        p = rv.cdf(z) * 2
    else:
        p = (1 - rv.cdf(z)) * 2
    print(f'p値は {p: .3f}')

In [16]:
pmean_test(sample, 130, 9)

帰無仮説を採択
p値は  0.053


In [17]:
# 正規分布の母分散の仮説検定を実装
# →母平均がある値σ^2ではないと主張するための検定
def pvar_test(sample, var0, alpha=0.05):
    u_var = np.var(sample, ddof=1)
    n = len(sample)
    rv = stats.chi2(df=n-1)
    interval = rv.interval(1-alpha)
    
    y = (n-1) * u_var / var0
    if interval[0] <= y <= interval[1]:
        print('帰無仮説を採択')
    else:
        print('帰無仮説を棄却')
    if y < rv.isf(0.5):
        p = rv.cdf(y) * 2
    else:
        p = (1 - rv.cdf(y)) * 2
    print(f'p値は{p: .3f}')

In [18]:
pvar_test(sample, 9)

帰無仮説を採択
p値は 0.085


In [19]:
# 正規分布の母平均の仮説検定を実装
# →母分散がわからない状況でのt検定

def pmean_test(sample, mean0, alpha=0.05):
    s_mean = np.mean(sample)
    u_var = np.var(sample, ddof=1)
    n = len(sample)
    rv = stats.t(df=n-1)
    interval = rv.interval(1-alpha)
    
    t = (s_mean - mean0) / np.sqrt(u_var / n)
    if interval[0] <= t <= interval[1]:
        print('帰無仮説を採択')
    else:
        print('帰無仮説を棄却')
    if t < 0:
        p = rv.cdf(t) * 2
    else:
        p = (1 - rv.cdf(t)) * 2
    print(f'p値は{p: .3f}')

In [20]:
pmean_test(sample, 130)

帰無仮説を採択
p値は 0.169


In [21]:
t, p = stats.ttest_1samp(sample, 130)
t, p

(-1.4551960206404198, 0.16933464230414275)

In [23]:
# データの対応がある→２つのデータが同一の個体を異なる条件で測定したか
training_rel = pd.read_csv('/Users/yuya/Desktop/Python_Static/python_stat_sample/data/ch11_training_rel.csv')
print(training_rel.shape)
training_rel.head()

(20, 2)


,前,後
0,59,41
1,52,63
2,55,68
3,61,59
4,59,84


In [26]:
# 筋トレで集中力が向上したかを調べる
# 筋トレ前後の集中力テストの平均点を比較する
training_rel['差'] = training_rel['後'] - training_rel['前']
training_rel.head()

,前,後,差
0,59,41,-18
1,52,63,11
2,55,68,13
3,61,59,-2
4,59,84,25


In [27]:
# 筋トレが集中力テストに与える影響がなければ、
# 筋トレ前後の平均点の差はランダムにばらつき平均0の分布になるはず
# ・帰無仮説: μdiff = 0
# ・対立仮説: μdeiff ≠ 0
#
# さらにこの差がそれぞれ独立に同一の正規分布に従っていると仮定できると、
# この検定は母分散がわからない場合の正規分布の母平均の推定
# 1標本のt検定に帰着できる

# 1標本のt検定を計算する
t, p = stats.ttest_1samp(training_rel['差'], 0)
p

0.04004419061842953

In [28]:
# 差をこちらで求めずに、関数におまかせ
t, p = stats.ttest_rel(training_rel['後'], training_rel['前'])
p

0.04004419061842953

In [29]:
# 対応のないt検定
# →データに対応がなく、2標本の母集団に正規分布を仮定できる場合の平均値の差の検定
#
# ・文化系の学生が多いAさんのクラス
# ・体育系の学生の多いBさんのクラス
# で集中力テストの平均点に差がでるか?
training_ind = pd.read_csv('/Users/yuya/Desktop/Python_Static/python_stat_sample/data/ch11_training_ind.csv')
print(training_ind.shape)
training_ind.head()

(20, 2)


,A,B
0,47,49
1,50,52
2,37,54
3,60,48
4,39,51


In [30]:
# ウェルチの方法で検定統計量tを求める
t, p = stats.ttest_ind(training_ind['A'], training_ind['B'], equal_var = False)
p

0.08695731107259361

In [31]:
# ウィルコクソンの順位検定
# →データに対応があるが、正規分布を仮定できない場合に、中央値の差を用いて検定する方法

# 上記の例で使用したデータと同様のデータで試す
training_rel = pd.read_csv('/Users/yuya/Desktop/Python_Static/python_stat_sample/data/ch11_training_rel.csv')
toy_df = training_rel[:6].copy()
toy_df

,前,後
0,59,41
1,52,63
2,55,68
3,61,59
4,59,84
5,45,37


In [33]:
# 対応のあるデータなので、データの差に着目
diff = toy_df['後'] - toy_df['前']
toy_df['差'] = diff
toy_df

,前,後,差
0,59,41,-18
1,52,63,11
2,55,68,13
3,61,59,-2
4,59,84,25
5,45,37,-8


In [34]:
# ウィルコクソンの順位検定では、名の通り順位によって検定を行う
# 1. 差の絶対値の小さい物から順に順位をつけていく
rank = stats.rankdata(abs(diff)).astype(int)
toy_df['順位'] = rank
toy_df

,前,後,差,順位
0,59,41,-18,5
1,52,63,11,3
2,55,68,13,4
3,61,59,-2,1
4,59,84,25,6
5,45,37,-8,2


In [35]:
# 2. ・差の符号がマイナスである物の順位の和
#     ・差の符号がプラスである物の順位の和
# を求める
r_minus = np.sum((diff < 0) * rank)
r_plus = np.sum((diff > 0) * rank)

r_minus, r_plus

# このr_minus, r_plusのうち、小さいほうが検定統計量になる

(8, 13)

In [38]:
# なぜこの方法で中央値の差の検定ができるか考察する
toy_df['後'] = toy_df['前'] + np.arange(1, 7)
diff = toy_df['後'] - toy_df['前']
rank = stats.rankdata(abs(diff)).astype(int)
toy_df['差'] = diff
toy_df['順位'] = rank
toy_df

,前,後,差,順位
0,59,60,1,1
1,52,54,2,2
2,55,58,3,3
3,61,65,4,4
4,59,64,5,5
5,45,51,6,6


In [40]:
# 差がマイナスの順位の和とプラスの順位の和を求める
r_minus = np.sum((diff < 0) * rank)
r_plus = np.sum((diff > 0) * rank)

r_minus, r_plus

# 差に偏りがあると検定統計量は0に近づく

(0, 21)

In [42]:
# 筋トレ後にテストの結果が上がった人、下がった人もいる場合を考える
toy_df['後'] = toy_df['前'] + [1, -2, -3, 4, 5, -6]
diff = toy_df['後'] - toy_df['前']
rank = stats.rankdata(abs(diff)).astype(int)
toy_df['差'] = diff
toy_df['順位'] = rank
toy_df

,前,後,差,順位
0,59,60,1,1
1,52,50,-2,2
2,55,52,-3,3
3,61,65,4,4
4,59,64,5,5
5,45,39,-6,6


In [43]:
# 差がマイナスの順位の和とプラスの順位の和を求める
r_minus = np.sum((diff < 0) * rank)
r_plus = np.sum((diff > 0) * rank)

r_minus, r_plus

(11, 10)

In [44]:
T, p = stats.wilcoxon(training_rel['前'], training_rel['後'])
p

0.03623390197753906

In [45]:
T, p = stats.wilcoxon(training_rel['後'] -  training_rel['前'])
p

0.039989471435546875

In [47]:
# 正規分布でもウィルコクソンの符号付き順位検定は使えるが、
# t検定に比べて検出力は下がる
n = 10000
diffs = np.round(stats.norm(3, 4).rvs(size=(n, 20)))

In [49]:
# t 検定
cnt = 0
alpha = 0.05
for diff in diffs:
    t, p = stats.ttest_1samp(diff, 0)
    if p < alpha:
        cnt += 1
cnt / n

0.883

In [52]:
# ウィルコクソンの符号付き順位検定
cnt = 0
alpha = 0.05
for diff in diffs:
    T, p = stats.wilcoxon(diff)
    if p < alpha:
        cnt += 1
cnt / n

0.873

In [54]:
# マン・ホイットニーのU検定(ウィルコクソンの順位和検定)
# →データに対応がなく、2標本の母集団に正規分布を仮定できない場合の中央値の検定
training_ind = pd.read_csv('/Users/yuya/Desktop/Python_Static/python_stat_sample/data/ch11_training_ind.csv')
toy_df = training_ind[:5].copy()
toy_df

,A,B
0,47,49
1,50,52
2,37,54
3,60,48
4,39,51


In [56]:
# U検定ではデータ全体に対して、値が小さい順に順位付け
rank = stats.rankdata(np.concatenate([toy_df['A'], toy_df['B']]))
rank_df = pd.DataFrame({'A': rank[:5],
                                            'B': rank[5:10]}).astype(int)
rank_df

,A,B
0,3,5
1,6,8
2,1,9
3,10,4
4,2,7


In [57]:
# 検定統計量としてAの順位の和を用いる(Bを用いてもよい)
n1 = len(rank_df['A'])
u = rank_df['A'].sum() - (n1*(n1+1))/2
u

7.0

In [58]:
# Aにいい順位がすべて集まった場合
rank_df = pd.DataFrame(np.arange(1, 11).reshape(2, 5).T,
                                              columns=['A', 'B'])
rank_df

,A,B
0,1,6
1,2,7
2,3,8
3,4,9
4,5,10


In [59]:
u = rank_df['A'].sum() - (n1*(n1+1))/2
u

0.0

In [61]:
# 逆に悪い順位が集まっていた場合
rank_df = pd.DataFrame(np.arange(1, 11).reshape(2, 5)[::-1].T,
                                            columns=['A', 'B'])
rank_df

,A,B
0,6,1
1,7,2
2,8,3
3,9,4
4,10,5


In [62]:
u = rank_df['A'].sum() - (n1*(n1+1))/2
u

25.0

In [64]:
# 既存の関数でU検定を行う
u, p = stats.mannwhitneyu(training_ind['A'], training_ind['B'],
                                                 alternative='two-sided')
p

0.05948611166127324